In [7]:
import cellpose
import torch 
import torchvision
import zarr
import numpy as np
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import pandas as pd
from dask.distributed import Client
import xarray as xr
import dask
import imageio

import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage

from cellpose import core, utils, io, models, metrics
from glob import glob

# model_type='cyto' or 'nuclei' or 'cyto2'
# start logger (to see training across epochs)
logger = io.logger_setup()



2022-08-03 13:16:03,360 [INFO] WRITING LOG OUTPUT TO /gpfs/commons/home/jsingh/.cellpose/run.log


In [8]:
from pathlib import Path

exp_dir = Path('/gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/results/20210323_4i4color/')
str(exp_dir/Path('final_zarr/m1a.zarr'))

'/gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/results/20210323_4i4color/final_zarr/m1a.zarr'

In [9]:
import yaml
with open('/gpfs/commons/home/jsingh/hudson/config/config.yaml') as f:
    config = yaml.safe_load(f)

In [10]:
config.update({'segmentation':{'marker':'LMNB1'}})

In [11]:
config

{'experiment_directory': '/nethome/kpandit/pyseq-image/src/demo/',
 'output_directory': '/nethome/kpandit/hudson/results/',
 'resources': {'partition': 'pi3',
  'time': '1:00:00',
  'mem': '16G',
  'dask_worker': {'log_directory': 'None'}},
 'stains': {1: {610: 'GFAP'},
  2: {740: 'ELAVL2'},
  3: {558: 'LMN1b'},
  4: {687: 'MBP'},
  5: {558: 'LMN1b_5', 610: 'GFAP_5', 687: 'MBP_5', 740: 'ELAVL2_5'}},
 'segmentation': {'marker': 'LMNB1'}}

In [13]:
import xarray as xr


### DELETE ME
from pathlib import Path
import yaml

exp_dir = Path('/gpfs/commons/groups/nygcfaculty/PySeq/20210323_4i4color/pipeline/results/20210323_4i4color/')
class snakemake():
    input = [str(exp_dir/Path('final_zarr/m1a.zarr'))]
    output = [str(exp_dir/Path('segmentation/m1a.tiff'))]
    with open('/gpfs/commons/home/jsingh/hudson/config/config.yaml') as f:
        config = yaml.safe_load(f)

### DELETE ME


# Open image from zarr store
image_path = Path(snakemake.input[0])
im_name = image_path.stem
image = xr.open_zarr(image_path).to_array()
image = image.squeeze().drop_vars('variable').rename(im_name)

# segment
logger = io.logger_setup()
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU,model_type='TN2')
#model = models.CellposeModel(model_type='TN2')
# Remove once priors steps in pipe
#one_z_plane = image.sel(obj_step = 8498, channel = 558, cycle=1)
arr = image.sel(snakemake.config.get('segmentation'))
channels = [0,0]
masks, flows, styles = model.eval(arr.values, diameter=None, channels=channels, cellprob_threshold= -6, flow_threshold= 1000)
imageio.imwrite(snakemake.output[0],masks)
    


2022-08-03 13:17:50,542 [INFO] WRITING LOG OUTPUT TO /gpfs/commons/home/jsingh/.cellpose/run.log
2022-08-03 13:17:50,640 [INFO] TORCH CUDA version not installed/working.
2022-08-03 13:17:50,643 [INFO] >> TN2 << model set to be used
2022-08-03 13:17:50,646 [INFO] >>>> using CPU
2022-08-03 13:17:51,133 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
